In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
import os
from skimage.io import imread
import numpy as np

In [15]:
all_images = glob(os.path.join('..', 'data', '*.tif'))
img_df = pd.DataFrame([dict(path = c_file) for c_file in all_images])
img_df['basename'] = img_df['path'].map(lambda x: os.path.basename(x).split('.')[0])
img_df['frame_id'] = img_df['basename'].map(lambda x: '_'.join(x.split('_')[0:-1]))
img_df['sample'] = img_df['basename'].map(lambda x: x.split('_')[0])
img_df['slice'] = img_df['basename'].map(lambda x: int(x.split('_')[3]))
img_df['frame'] = img_df['basename'].map(lambda x: int(x.split('_')[1]))
# group by prefix
img_3d_df = img_df.sort_values(['sample', 'frame_id', 'slice']).groupby(['sample', 'frame', 'frame_id'])['path'].apply(list).reset_index()
img_3d_df

,sample,frame,frame_id,path
0,MI5,0,MI5_0_00,"[../data/MI5_0_00_1500.rec.8bit.tif, ../data/M..."
1,MI5,1,MI5_1_00,"[../data/MI5_1_00_1500.rec.8bit.tif, ../data/M..."
2,MI5,1,MI5_1_01,"[../data/MI5_1_01_1500.rec.8bit.tif, ../data/M..."
3,MI5,1,MI5_1_02,"[../data/MI5_1_02_1500.rec.8bit.tif, ../data/M..."
4,MI7,0,MI7_0_00,"[../data/MI7_0_00_1300.rec.8bit.tif, ../data/M..."
5,MI7,1,MI7_1_00,"[../data/MI7_1_00_1300.rec.8bit.tif, ../data/M..."
6,MI7,1,MI7_1_01,"[../data/MI7_1_01_1300.rec.8bit.tif, ../data/M..."
7,MI7,1,MI7_1_02,"[../data/MI7_1_02_1300.rec.8bit.tif, ../data/M..."


In [17]:
from skimage.filters import threshold_minimum
from skimage.morphology.convex_hull import convex_hull_image
from skimage.morphology import binary_closing, binary_opening, disk, label
from skimage.measure import regionprops

from skimage.color import label2rgb
from scipy import ndimage as ndi

def threshold_slice_2d(raw_slice):
    in_slice = ndi.median_filter(raw_slice, [7, 7])
    # basic threshold
    thresh_min = threshold_minimum(in_slice)
    # make solid component
    solid_part = binary_closing(in_slice>thresh_min, disk(4))
    solid_part = ndi.binary_fill_holes(solid_part)
    # calculate porous component inside solid
    porous_part = (in_slice < thresh_min)*solid_part
    #porous_part = binary_opening(porous_part, disk(2))
    porous_part = binary_closing(porous_part, disk(2))
    # create label by combining
    label_slice = solid_part.astype(np.uint8).clip(0,1)
    label_slice += (porous_part).astype(np.uint8).clip(0,1)
    return label_slice, thresh_min

def paths_to_vol(in_paths):
    return np.stack([threshold_slice_2d(imread(x))[0] for x in in_paths],0)

In [21]:
import ipywidgets as ipw
from ipyvolume import quickvolshow

# MI5

In [22]:
_, c_row = next(img_3d_df.query('sample == "MI5"').iterrows())
n_vol = paths_to_vol(c_row['path'])
quickvolshow(n_vol)

/srv/conda/lib/python3.6/site-packages/ipyvolume/serialize.py:66: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


VBox(children=(VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step=0.001), FloatSlider(value=0.5, max=1.0, step=0.001), FloatSlider(value=0.9, max=1.0, step=0.001))), HBox(children=(Label(value='opacities:'), FloatSlider(value=0.01, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001))))), Figure(ambient_coefficient=1.0, camera_center=[0.0, 0.0, 0.0], data_max=2.0, diffuse_coefficient=0.0, height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], specular_coefficient=0.0, style={'axes': {'visible': True, 'label': {'color': 'black'}, 'ticklabel': {'color': 'black'}, 'color': 'black'}, 'box': {'visible': True}, 'background-color': 'white'}, tf=TransferFunctionWidgetJs3(level3=0.9), volume_data=array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8), width=400, xlim=[0.0, 1.0], ylim=[0.0, 1.0], zlim=[0.0, 1.0])))

# MI7

In [23]:
_, c_row = next(img_3d_df.query('sample == "MI7"').iterrows())
n_vol = paths_to_vol(c_row['path'])
quickvolshow(n_vol, lighting=True)

/srv/conda/lib/python3.6/site-packages/ipyvolume/serialize.py:66: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


VBox(children=(VBox(children=(HBox(children=(Label(value='levels:'), FloatSlider(value=0.1, max=1.0, step=0.001), FloatSlider(value=0.5, max=1.0, step=0.001), FloatSlider(value=0.9, max=1.0, step=0.001))), HBox(children=(Label(value='opacities:'), FloatSlider(value=0.01, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001), FloatSlider(value=0.05, max=0.2, step=0.001))))), Figure(camera_center=[0.0, 0.0, 0.0], data_max=2.0, height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], style={'axes': {'visible': True, 'label': {'color': 'black'}, 'ticklabel': {'color': 'black'}, 'color': 'black'}, 'box': {'visible': True}, 'background-color': 'white'}, tf=TransferFunctionWidgetJs3(level3=0.9), volume_data=array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint8), width=400, xlim=[0.0, 1.0], ylim=[0.0, 1.0], zlim=[0.0, 1.0]), HBox(children=(FloatSlider(value=0.5, description='ambient', max=1.0, step=0.001), FloatSlider(value=0.8, description='diffuse', max=1.0, step=0.001))), HBox(children=(FloatSlider(value=0.5, description='specular', max=1.0, step=0.001), FloatSlider(value=5.0, description='specular exp', max=10.0, step=0.001)))))